### spam detector

In [60]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
import re

In [61]:
df = pd.read_csv('youtube.csv')
df.head()

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ...",1
1,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07T12:37:15,Hey guys check out my new channel and our firs...,1
2,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com,1
3,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .﻿,1


In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   COMMENT_ID  350 non-null    object
 1   AUTHOR      350 non-null    object
 2   DATE        350 non-null    object
 3   CONTENT     350 non-null    object
 4   CLASS       350 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 13.8+ KB


In [63]:
# Link içeren yorumları tespit etme ve yeni bir sütun oluşturma
def contains_link(text):

    url_pattern = r'https?://\S+|www\.\S+'
    return bool(re.search(url_pattern, text))


In [64]:
df['contains_link'] = df['CONTENT'].apply(contains_link)

In [65]:
# Metin Temizliği
def clean_text(text):
    text = text.lower()
    text = re.sub(r'https?://\S+|www\.\S+', '', text)  # URL'leri kaldır
    text = re.sub(r'[^a-z0-9\s]', '', text)  # Noktalama işaretlerini ve özel karakterleri kaldır
    return text

In [66]:
df['cleaned_content'] = df['CONTENT'].apply(clean_text)

In [67]:
df[['cleaned_content', 'contains_link']].sample(10)

,cleaned_content,contains_link
234,what can i saythis song he just change the wor...,False
122,suscribe my channel please,False
225,you think youre smart headbutt your face,False
145,i think this is now a place to promote channel...,False
96,i made a gaming channel unique right l angry m...,False
246,remove this video its wank,False
46,now its 1884034783 views pls comment the view ...,False
15,hello do you like gaming art videos scientific...,False
346,i dont now why im watching this in 2014,False
99,,True


In [68]:
x = df[['cleaned_content', 'contains_link']]
y = df['CLASS']

In [69]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [70]:
preprocessor = ColumnTransformer(
    transformers=[
        ('tfidf', TfidfVectorizer(), 'cleaned_content'),
        ('link', OneHotEncoder(), ['contains_link'])
    ])

In [71]:
model_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', MultinomialNB())

])

eğitim ve test olarak ayırma

In [80]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=41)


model eğitimi

In [81]:
model_pipeline.fit(x_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('tfidf', TfidfVectorizer(),
                                                  'cleaned_content'),
                                                 ('link', OneHotEncoder(),
                                                  ['contains_link'])])),
                ('classifier', MultinomialNB())])

In [82]:
predictions = model_pipeline.predict(x_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       1.00      0.89      0.94        82
           1       0.87      1.00      0.93        58

    accuracy                           0.94       140
   macro avg       0.93      0.95      0.93       140
weighted avg       0.94      0.94      0.94       140



In [83]:
print("Doğruluk:", accuracy_score(y_test, predictions))

Doğruluk: 0.9357142857142857
